In [ ]:
%load_ext autoreload
%autoreload 2

# Augment common non-negatives

Add triplets and name variants and nicknames to common non-negatives

In [ ]:
import pandas as pd
from tqdm.auto import tqdm

from src.data.utils import read_csv

In [ ]:
given_surname = "given"

common_non_negatives_path = f"../references/common_{given_surname}_non_negatives.csv"
triplets_path=f"../data/processed/tree-hr-{given_surname}-triplets-v2-1000.csv.gz"
name_variants_path = f"../references/{given_surname}_variants.csv"
given_nicknames_path = "../references/givenname_nicknames.csv"

augmented_path = f"../data/processed/common_{given_surname}_non_negatives-augmented.csv"

## Load data

### read common non-negatives

In [ ]:
common_non_negatives_df = read_csv(common_non_negatives_path)
print(len(common_non_negatives_df))
common_non_negatives_df.head(3)

In [ ]:
common_non_negatives = set()
for name1, name2 in common_non_negatives_df.values.tolist():
    common_non_negatives.add((name1, name2))
    common_non_negatives.add((name2, name1))
len(common_non_negatives)

### add triplets

In [ ]:
triplets_df = pd.read_csv(triplets_path)
print(len(triplets_df))
triplets_df.head(3)

In [ ]:
for anchor, pos, neg in zip(
    triplets_df['anchor'], 
    triplets_df['positive'],
    triplets_df['negative'],
):
    common_non_negatives.add((anchor, pos))
    common_non_negatives.add((pos, anchor))
    common_non_negatives.add((anchor, neg))
    common_non_negatives.add((neg, anchor))
len(common_non_negatives)

### add name variants

In [ ]:
name_variants_df = read_csv(name_variants_path)
print(len(name_variants_df))
name_variants_df.head(3)

In [ ]:
for name1, name2 in name_variants_df.values.tolist():
    common_non_negatives.add((name1, name2))
    common_non_negatives.add((name2, name1))
len(common_non_negatives)

### add given nicknames

In [ ]:
if given_surname == "given":
    with open(given_nicknames_path, "rt") as f:
        for line in f.readlines():
            names = line.split(',')
            for name1 in names:
                for name2 in names:
                    if name1 > name2:
                        common_non_negatives.add((name1, name2))
                        common_non_negatives.add((name2, name1))
len(common_non_negatives)

## Save augmented non-negatives

In [ ]:
records = []
for name1, name2 in common_non_negatives:
    records.append({'name1': name1, 'name2': name2})
df = pd.DataFrame(records)
df.to_csv(augmented_path, index=False)

## Miscellaneous

Generate common non-negatives from existing standard